# Β1.ii) Ταξινόμηση με Logistic Regression και Word2Vec

Αυτό το notebook εκτελεί ταξινόμηση με Logistic Regression χρησιμοποιώντας Word2Vec για την εξαγωγή χαρακτηριστικών από κείμενο.

**Κύρια Χαρακτηριστικά:**
- **Μοντέλο**: Logistic Regression με liblinear solver
- **Feature Extraction**: Word2Vec embeddings με document-level aggregation
- **Dataset**: "greek_legal_code" από το Hugging Face Hub

## Τεχνικές Λεπτομέρειες Logistic Regression

### Logistic Regression Παράμετροι:

**Solver**: `liblinear`
- Κατάλληλος για μικρά προς μεσαία datasets
- Υποστηρίζει L1 και L2 regularization
- Αποδοτικός για binary και multiclass classification

**Max Iterations**: `1000`
- Μέγιστος αριθμός επαναλήψεων για convergence
- Εξασφαλίζει ότι ο αλγόριθμος θα τερματίσει

**Random State**: `42`
- Εξασφαλίζει αναπαραγωγιμότητα των αποτελεσμάτων

### Μαθηματικό Υπόβαθρο:

**Logistic Function:**
- P(y=1|x) = 1 / (1 + e^(-z))
- όπου z = β₀ + β₁x₁ + β₂x₂ + ... + βₙxₙ

**Cost Function (Log-Likelihood):**
- J(θ) = -1/m * Σ[y·log(h(x)) + (1-y)·log(1-h(x))]

**Πλεονεκτήματα:**
- Γρήγορη εκπαίδευση και πρόβλεψη
- Παρέχει πιθανότητες κλάσεων
- Δεν απαιτεί feature scaling (αλλά βοηθάει)
- Ερμηνεύσιμα coefficients

---

In [ ]:
import sys
import os
import numpy as np
import time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from datasets import load_dataset
from gensim.models import Word2Vec
import json

# Προσθήκη του parent directory στο path για να μπορούμε να εισάγουμε το utils
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# Import utils functions
try:
    from utils import (
        load_and_preprocess_data,
        run_experiment,
        script_execution_timer
    )
    print("Utils imported successfully")
except ImportError as e:
    print(f"Error importing utils: {e}")
    print("Continuing without utils functions...")

## Παραμετροποίηση

Ορισμός των παραμέτρων για το πείραμα Logistic Regression με Word2Vec.

In [ ]:
# Κοινές παράμετροι
DATASET_CONFIG = "subject"  # επιλογές: "volume", "chapter", "subject"
SUBSET_PERCENTAGE = 1.0     # 1.0 για όλα τα δεδομένα, <1.0 για υποσύνολο
N_SPLITS_CV = 1             # 1 για single split, >1 για K-Fold CV
RANDOM_STATE = 42
SINGLE_SPLIT_TEST_SIZE = 0.2

# Feature Engineering
SAVE_TRAINED_FEATURE_MODELS = True
LOAD_TRAINED_FEATURE_MODELS_IF_EXIST = True

# Word2Vec παράμετροι
WORD2VEC_VECTOR_SIZE = 100   # Διάσταση των word embeddings
WORD2VEC_WINDOW = 5          # Μέγεθος παραθύρου context
WORD2VEC_MIN_COUNT = 2       # Ελάχιστη συχνότητα λέξης
WORD2VEC_WORKERS = 4         # Αριθμός CPU cores
WORD2VEC_SG = 0              # 0 για CBOW, 1 για Skip-gram
WORD2VEC_EPOCHS = 10         # Εποχές εκπαίδευσης

print(f"Dataset config: {DATASET_CONFIG}")
print(f"Subset percentage: {SUBSET_PERCENTAGE}")
print(f"Word2Vec vector size: {WORD2VEC_VECTOR_SIZE}")
print(f"Word2Vec window: {WORD2VEC_WINDOW}")
print(f"Word2Vec algorithm: {'Skip-gram' if WORD2VEC_SG else 'CBOW'}")

## Τεχνικά Χαρακτηριστικά Word2Vec

### Word2Vec Architecture:

**CBOW (Continuous Bag of Words)** - Προεπιλογή:
- **Πρόβλεψη**: Προβλέπει την κεντρική λέξη από το context
- **Αρχιτεκτονική**: Input → Hidden Layer → Output
- **Ταχύτητα**: Γρηγορότερη εκπαίδευση
- **Απόδοση**: Καλύτερη σε συχνές λέξεις

**Skip-gram** - Εναλλακτική:
- **Πρόβλεψη**: Προβλέπει το context από την κεντρική λέξη  
- **Αρχιτεκτονική**: Input → Hidden Layer → Multiple Outputs
- **Ταχύτητα**: Βραδύτερη εκπαίδευση
- **Απόδοση**: Καλύτερη σε σπάνιες λέξεις και μικρά datasets

### Παράμετροι Word2Vec:

- **Vector Size (100)**: Διάσταση του embedding space
- **Window (5)**: Context window - λέξεις πριν και μετά την target
- **Min Count (2)**: Φιλτράρισμα σπάνιων λέξεων (<2 εμφανίσεις)
- **Workers (4)**: Παραλληλοποίηση για ταχύτητα
- **Epochs (10)**: Πολλαπλά περάσματα από το corpus

### Document-Level Feature Extraction:

**Aggregation Methods:**
1. **Mean Pooling**: Μέσος όρος όλων των word vectors (συνήθης επιλογή)
2. **Max Pooling**: Μέγιστη τιμή ανά διάσταση
3. **Weighted Average**: Σταθμισμένος μέσος (π.χ. με TF-IDF weights)

**Πλεονεκτήματα Word2Vec:**
- Καταγράφει σημασιολογικές σχέσεις μεταξύ λέξεων
- Dense representations (αντί για sparse BoW/TF-IDF)
- Χαμηλότερη διάσταση από vocabulary-based methods
- Μπορεί να γενικεύσει σε νέες λέξεις με παρόμοια contexts

**Μειονεκτήματα:**
- Χρειάζεται αρκετά δεδομένα για καλή εκπαίδευση
- Δεν κατανοεί τη σειρά των λέξεων στο document
- Aggregation μπορεί να χάσει σημαντική πληροφορία

## Φόρτωση και Προεπεξεργασία Δεδομένων

Φόρτωση του dataset και προετοιμασία για το πείραμα.

In [ ]:
# Φόρτωση των δεδομένων
print("Φόρτωση δεδομένων...")

# Αν υπάρχει η συνάρτηση utils, χρησιμοποιούμε αυτή
if 'load_and_preprocess_data' in globals():
    all_texts, all_labels, label_names, num_classes = load_and_preprocess_data(
        DATASET_CONFIG, SUBSET_PERCENTAGE, RANDOM_STATE
    )
else:
    # Εναλλακτική φόρτωση χωρίς utils
    print("Φόρτωση dataset χωρίς utils...")
    try:
        ds = load_dataset("AI-team-UoA/greek_legal_code", DATASET_CONFIG, trust_remote_code=True)
        dataset_split = ds['train']
        all_texts = dataset_split['text']
        all_labels = np.array(dataset_split['label'])
        label_names = dataset_split.features['label'].names
        num_classes = len(label_names)
        print(f"Dataset loaded successfully. Samples: {len(all_texts)}, Classes: {num_classes}")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        all_texts, all_labels, label_names, num_classes = None, None, None, None

if all_texts is not None:
    print(f"Συνολικά samples: {len(all_texts)}")
    print(f"Αριθμός κλάσεων: {num_classes}")
    print(f"Ονόματα κλάσεων: {label_names}")
    print(f"Παράδειγμα κειμένου: {all_texts[0][:200]}...")

## Logistic Regression με Word2Vec

### Διαδικασία Feature Extraction:

1. **Tokenization**: Διαχωρισμός κειμένων σε λέξεις
2. **Word2Vec Training**: Εκπαίδευση embeddings στο corpus
3. **Document Vectorization**: Μετατροπή κάθε εγγράφου σε vector μέσω mean pooling
4. **Classification**: Εκπαίδευση Logistic Regression στα document vectors

Εκτέλεση ταξινόμησης με Logistic Regression χρησιμοποιώντας Word2Vec για την εξαγωγή χαρακτηριστικών.

In [ ]:
def run_logistic_regression_word2vec_experiment():
    """Εκτέλεση Logistic Regression με Word2Vec"""
    
    print("=== Ξεκινάει το πείραμα Logistic Regression με Word2Vec ===")
    start_time = time.time()
    
    # Παραμετροποίηση
    model_name_script = "LogRegr"
    feature_method_name = "Word2Vec"
    base_run_id = f"{model_name_script}_{feature_method_name}_{DATASET_CONFIG}"
    
    # Ρύθμιση feature extraction
    feature_config = {
        'method': 'word2vec',
        'params': {
            'vector_size': WORD2VEC_VECTOR_SIZE,
            'window': WORD2VEC_WINDOW,
            'min_count': WORD2VEC_MIN_COUNT,
            'workers': WORD2VEC_WORKERS,
            'sg': WORD2VEC_SG,
            'epochs': WORD2VEC_EPOCHS
        }
    }
    
    # Ρύθμιση μοντέλου
    model_class = LogisticRegression
    model_init_params = {
        'solver': 'liblinear', 
        'max_iter': 1000, 
        'random_state': RANDOM_STATE
    }
    
    # Δημιουργία output directories
    script_dir = os.path.dirname(os.path.abspath(''))
    experiment_output_base_dir = os.path.join(script_dir, "outputs", base_run_id)
    reports_output_dir = os.path.join(experiment_output_base_dir, "reports")
    feature_models_output_dir = os.path.join(experiment_output_base_dir, "feature_models")
    
    # Δημιουργία directories αν δεν υπάρχουν
    os.makedirs(reports_output_dir, exist_ok=True)
    os.makedirs(feature_models_output_dir, exist_ok=True)
    
    print(f"Output directory: {experiment_output_base_dir}")
    
    # Αν υπάρχει η run_experiment από utils, χρησιμοποιούμε αυτή
    if 'run_experiment' in globals() and all_texts is not None:
        try:
            run_experiment(
                all_texts=all_texts,
                all_labels=all_labels,
                label_names=label_names,
                num_classes=num_classes,
                feature_config=feature_config,
                model_class=model_class,
                model_init_params=model_init_params,
                n_splits_cv=N_SPLITS_CV,
                single_split_test_size=SINGLE_SPLIT_TEST_SIZE,
                random_state=RANDOM_STATE,
                reports_output_dir=reports_output_dir,
                feature_models_output_dir=feature_models_output_dir,
                save_trained_feature_models=SAVE_TRAINED_FEATURE_MODELS,
                load_trained_feature_models_if_exist=LOAD_TRAINED_FEATURE_MODELS_IF_EXIST
            )
        except Exception as e:
            print(f"Error in run_experiment: {e}")
            print("Εκτέλεση απλουστευμένου πειράματος...")
            run_simple_word2vec_experiment(feature_config, model_class, model_init_params)
    else:
        print("Εκτέλεση απλουστευμένου πειράματος...")
        run_simple_word2vec_experiment(feature_config, model_class, model_init_params)
    
    end_time = time.time()
    print(f"=== Ολοκληρώθηκε το πείραμα σε {end_time - start_time:.2f} δευτερόλεπτα ===")

# Εκτέλεση πειράματος
if all_texts is not None:
    run_logistic_regression_word2vec_experiment()
else:
    print("Δεν μπορεί να εκτελεστεί το πείραμα - δεν φορτώθηκαν δεδομένα")

## Απλουστευμένη Εκτέλεση Πειράματος

Στην περίπτωση που δεν είναι διαθέσιμες οι συναρτήσεις utils, εκτελείται μια απλουστευμένη έκδοση.

In [ ]:
def simple_tokenize(text):
    """Απλή tokenization για Word2Vec"""
    return text.lower().split()

def train_word2vec_model(texts, params):
    """Εκπαίδευση Word2Vec μοντέλου"""
    print("Tokenization κειμένων...")
    tokenized_texts = [simple_tokenize(text) for text in texts]
    
    print("Εκπαίδευση Word2Vec μοντέλου...")
    model = Word2Vec(
        sentences=tokenized_texts,
        vector_size=params['vector_size'],
        window=params['window'],
        min_count=params['min_count'],
        workers=params['workers'],
        sg=params['sg'],
        epochs=params['epochs']
    )
    
    print(f"Word2Vec vocabulary size: {len(model.wv.key_to_index)}")
    return model

def document_to_vector(doc, w2v_model, vector_size):
    """Μετατροπή εγγράφου σε vector μέσω mean pooling"""
    tokens = simple_tokenize(doc)
    word_vectors = []
    
    for token in tokens:
        if token in w2v_model.wv:
            word_vectors.append(w2v_model.wv[token])
    
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        # Αν δεν υπάρχουν γνωστές λέξεις, επιστροφή μηδενικού vector
        return np.zeros(vector_size)

def run_simple_word2vec_experiment(feature_config, model_class, model_init_params):
    """Απλουστευμένη εκτέλεση πειράματος χωρίς utils"""
    
    if all_texts is None or all_labels is None:
        print("Δεν είναι διαθέσιμα δεδομένα για το πείραμα")
        return
    
    print("Εκτέλεση απλουστευμένου πειράματος Word2Vec")
    
    # Διαχωρισμός σε training/testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        all_texts, all_labels, 
        test_size=SINGLE_SPLIT_TEST_SIZE, 
        random_state=RANDOM_STATE,
        stratify=all_labels
    )
    
    print(f"Training samples: {len(X_train)}")
    print(f"Testing samples: {len(X_test)}")
    
    # Εκπαίδευση Word2Vec
    w2v_params = feature_config['params']
    w2v_model = train_word2vec_model(X_train, w2v_params)
    
    # Μετατροπή εγγράφων σε vectors
    print("Μετατροπή training documents σε vectors...")
    X_train_vectors = np.array([
        document_to_vector(doc, w2v_model, w2v_params['vector_size']) 
        for doc in X_train
    ])
    
    print("Μετατροπή testing documents σε vectors...")
    X_test_vectors = np.array([
        document_to_vector(doc, w2v_model, w2v_params['vector_size']) 
        for doc in X_test
    ])
    
    print(f"Feature shape: {X_train_vectors.shape}")
    
    # Model training
    print("Εκπαίδευση Logistic Regression μοντέλου...")
    model = model_class(**model_init_params)
    model.fit(X_train_vectors, y_train)
    
    # Predictions
    print("Υπολογισμός προβλέψεων...")
    y_pred = model.predict(X_test_vectors)
    
    # Evaluation
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=label_names)
    
    print(f"\n=== Αποτελέσματα για Logistic Regression + Word2Vec ===")
    print(f"Accuracy: {accuracy:.4f}")
    print("\nClassification Report:")
    print(report)
    
    # Επιπλέον πληροφορίες
    print(f"\nWord2Vec Info:")
    print(f"- Vocabulary size: {len(w2v_model.wv.key_to_index)}")
    print(f"- Vector dimensions: {w2v_params['vector_size']}")
    print(f"- Training algorithm: {'Skip-gram' if w2v_params['sg'] else 'CBOW'}")
    
    return {
        'accuracy': accuracy,
        'classification_report': report,
        'w2v_vocab_size': len(w2v_model.wv.key_to_index),
        'method': 'LogisticRegression + Word2Vec'
    }

print("Συνάρτηση απλουστευμένου πειράματος ορίστηκε επιτυχώς")

## Εντολές Εκτέλεσης

Για να εκτελέσετε το πείραμα χωριστά μέσω terminal, χρησιμοποιήστε:

```bash
# Για Logistic Regression με Word2Vec
python merosB1/ii/main_LogRegr_W2V.py
```

Τα αποτελέσματα θα αποθηκευτούν στον φάκελο `outputs/`.

In [ ]:
# Εναλλακτική εκτέλεση με υποσύνολο δεδομένων για γρήγορη δοκιμή
print("\n=== Γρήγορη Δοκιμή με Υποσύνολο Δεδομένων ===")

if all_texts is not None and len(all_texts) > 500:
    # Χρήση 5% των δεδομένων για γρήγορη δοκιμή
    subset_size = max(100, int(len(all_texts) * 0.05))  # Τουλάχιστον 100 samples
    indices = np.random.choice(len(all_texts), subset_size, replace=False)
    
    subset_texts = [all_texts[i] for i in indices]
    subset_labels = all_labels[indices]
    
    print(f"Χρήση {subset_size} samples από {len(all_texts)} για γρήγορη δοκιμή")
    
    # Διαχωρισμός δεδομένων
    X_train, X_test, y_train, y_test = train_test_split(
        subset_texts, subset_labels, test_size=0.2, random_state=42, stratify=subset_labels
    )
    
    # Γρήγορη εκπαίδευση Word2Vec με μικρότερες παραμέτρους
    quick_w2v_params = {
        'vector_size': 50,   # Μικρότερο vector size
        'window': 3,         # Μικρότερο window
        'min_count': 1,      # Χαμηλότερο min_count
        'workers': 2,        # Λιγότεροι workers
        'sg': 0,             # CBOW
        'epochs': 5          # Λιγότερες εποχές
    }
    
    print("Γρήγορη εκπαίδευση Word2Vec...")
    w2v_model = train_word2vec_model(X_train, quick_w2v_params)
    
    # Feature extraction
    X_train_vectors = np.array([
        document_to_vector(doc, w2v_model, quick_w2v_params['vector_size']) 
        for doc in X_train
    ])
    X_test_vectors = np.array([
        document_to_vector(doc, w2v_model, quick_w2v_params['vector_size']) 
        for doc in X_test
    ])
    
    # Quick training
    lr_model = LogisticRegression(solver='liblinear', max_iter=500, random_state=42)
    lr_model.fit(X_train_vectors, y_train)
    
    y_pred = lr_model.predict(X_test_vectors)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"\nΓρήγορη δοκιμή - Accuracy: {accuracy:.4f}")
    print(f"Word2Vec vocabulary: {len(w2v_model.wv.key_to_index)} λέξεις")
    print(f"Feature dimensions: {quick_w2v_params['vector_size']}")
    
else:
    print("Δεν υπάρχουν αρκετά δεδομένα για δοκιμή")

## Σύγκριση με BoW/TF-IDF Μεθόδους

### Word2Vec vs Traditional Methods:

| Κριτήριο | Word2Vec | BoW/TF-IDF |
|----------|----------|------------|
| **Διάσταση Features** | Χαμηλή (100-300) | Υψηλή (χιλιάδες) |
| **Σημασιολογική Πληροφορία** | Ναι | Όχι |
| **Συχνότητα Λέξεων** | Όχι άμεσα | Ναι |
| **Αραιότητα** | Dense vectors | Sparse vectors |
| **Υπολογιστική Πολυπλοκότητα** | Μέτρια (training) | Χαμηλή |
| **Generalization** | Καλή | Περιορισμένη |
| **Interpretability** | Μέτρια | Υψηλή |

### Αναμενόμενα Αποτελέσματα:

**Word2Vec αναμένεται να υπερτερεί όταν:**
- Το corpus έχει πλούσιο vocabulary
- Υπάρχουν σημασιολογικές σχέσεις μεταξύ λέξεων
- Το dataset είναι αρκετά μεγάλο για καλή εκπαίδευση embeddings

**BoW/TF-IDF μπορεί να είναι καλύτερο όταν:**
- Η συχνότητα λέξεων είναι κρίσιμη
- Το dataset είναι μικρό
- Χρειάζεται γρήγορη εκτέλεση

### Βελτιστοποιήσεις Word2Vec:

1. **Hyperparameter Tuning:**
   - Vector size: δοκιμή 100, 200, 300
   - Window size: 3-10 ανάλογα με το είδος κειμένου
   - Min count: ισορροπία μεταξύ vocabulary size και coverage

2. **Preprocessing:**
   - Καλύτερη tokenization (αφαίρεση σημείων στίξης)
   - Stemming/Lemmatization για ελληνικά
   - Αφαίρεση stop words

3. **Aggregation Methods:**
   - Weighted average με TF-IDF weights
   - Doc2Vec για document-level embeddings
   - Attention-based pooling

## Πρακτικές Συμβουλές για Βελτίωση

### Προεπεξεργασία Κειμένου:
- **Tokenization**: Χρήση πιο εξελιγμένων tokenizers
- **Normalization**: Μετατροπή σε πεζά, αφαίρεση ειδικών χαρακτήρων
- **Stop Words**: Αφαίρεση συχνών λέξεων χωρίς σημασιολογική αξία
- **Stemming**: Μείωση λέξεων στη ρίζα τους (για ελληνικά: greek stemmer)

### Βελτιστοποίηση Word2Vec:
- **Vector Size**: Μεγαλύτερα vectors για πλουσιότερη αναπαράσταση
- **Training Data**: Περισσότερα δεδομένα για καλύτερα embeddings
- **Pre-trained Embeddings**: Χρήση προ-εκπαιδευμένων Greek embeddings
- **Sub-word Models**: FastText για handling OOV words

### Μοντέλο Optimization:
- **Regularization**: L1/L2 για αποφυγή overfitting
- **Cross-validation**: K-fold για αξιόπιστα αποτελέσματα
- **Feature Engineering**: Συνδυασμός Word2Vec με άλλα features
- **Ensemble Methods**: Συνδυασμός με άλλους classifiers

### Αξιολόγηση:
- **Confusion Matrix**: Ανάλυση λαθών ανά κατηγορία
- **Feature Analysis**: Εύρεση των πιο σημαντικών dimensions
- **Learning Curves**: Έλεγχος για overfitting/underfitting

## Αναμενόμενα Αποτελέσματα βάσει των προηγούμενων εκτελέσεων

Βάσει των αρχείων στο `outputsfinal/ii/`, αναμένουμε:

- **Γενικά καλή απόδοση** σε σχέση με BoW/TF-IDF μεθόδους
- **Βελτίωση στη generalization** λόγω semantic embeddings
- **Καλύτερα αποτελέσματα σε `subject` configuration** 
- **Πιθανή υστέρηση σε μικρά datasets** λόγω της πολυπλοκότητας του Word2Vec
- **Trade-off μεταξύ ταχύτητας και απόδοσης** σε σχέση με απλούστερες μεθόδους

---

## Σύνοψη Αποτελεσμάτων

Τα αποτελέσματα του πειράματος θα αποθηκευτούν στον φάκελο `outputs/` και θα περιλαμβάνουν:

1. **Logistic Regression με Word2Vec**: Classification reports και μετρικές απόδοσης
2. **Εκπαιδευμένο Word2Vec μοντέλο**: Για μελλοντική χρήση
3. **Feature vectors**: Document-level representations

Κάθε πείραμα παράγει:
- Classification report (σε μορφή JSON και text)
- Αποθηκευμένα μοντέλα (Word2Vec και Logistic Regression)
- Λεπτομερή logs της εκτέλεσης
- Στατιστικά για το Word2Vec vocabulary

Η σύγκριση με τις μεθόδους του Β1.i θα δείξει τα πλεονεκτήματα και μειονεκτήματα των semantic embeddings έναντι των frequency-based μεθόδων.